In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [11]:
base_html = 'C://Users//bruno//Documents//DS//DataSets Grandes//Historicos Economicos//MEP Historico.html'

Fuente: https://www.ambito.com/contenidos/dolar-mep-historico.html

In [3]:
mep_hist = pd.read_html(base_html) # this parses all the tables in webpages to a list
df_MEP = mep_hist[0]
df_MEP.head() # Values are int without decimals, which is incorrect due to the real value's nature

,Fecha,Referencia
0,09-07-2021,16642
1,08-07-2021,16642
2,07-07-2021,16620
3,06-07-2021,16598
4,05-07-2021,16645


In [6]:
new_value_list = []
for value in df_MEP.Referencia:
    ss1, ss2 = str(value)[:2], str(value)[2:] # Slicing the value as string to get separated the number
    new = ss1+'.'+ss2 # Adding the . into the value
    new_value_list.append(float(new)) # Adding the new value to a list}
df_MEP.Referencia = pd.Series(new_value_list) # Converting it to a Series and inserting replacing the extracted values

In [8]:
df_MEP.tail(5) # Checking last values with changes made

,Fecha,Referencia
472,24-03-2020,91.4
473,23-03-2020,91.4
474,22-03-2020,91.4
475,21-03-2020,91.4
476,20-03-2020,91.4


In [9]:
df_MEP.to_csv('../DataSets Grandes/Historicos Economicos/MEP.csv')